# Qwen-Audio-Chat

Following https://huggingface.co/Qwen/Qwen-Audio-Chat

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import torch
torch.manual_seed(1234)



In [ ]:
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-Audio-Chat", trust_remote_code=True)

# use bf16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio-Chat", device_map="auto", trust_remote_code=True, bf16=True).eval()
# use fp16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio-Chat", device_map="auto", trust_remote_code=True, fp16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio-Chat", device_map="cpu", trust_remote_code=True).eval()
# use cuda device
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-Audio-Chat", device_map="cuda", trust_remote_code=True).eval()

# Specify hyperparameters for generation (No need to do this if you are using transformers>4.32.0)
# model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-Audio-Chat", trust_remote_code=True)

## Examples

## Our data

In [6]:
query = tokenizer.from_list_format([
    {'audio': 'audio/fatalvapes_asmr_narration.mp4'}, # Either a local path or an url
    {'text': 'what does the person say?'},
])
response, history = model.chat(tokenizer, query=query, history=None)
print(response)

The person says in English: "Masking X-pan Lush Ice".


In [8]:
# 2nd dialogue turn
response, history = model.chat(tokenizer, 'Is there rhythmic tapping or noises in the background?', history=history)
print(response)

gen:Yes, there is rhythmic tapping or noises in the background.


In [9]:
response, history = model.chat(tokenizer, 'Is there rhythmic tapping or noises in the background? Rate your certainty in a probability between 0% and 100%', history=history)
print(response)

gen: 100%
